## This is for now not working. Regression patients are given differently than deepmil patients.

In [39]:
import pandas as pd
from sklearn import metrics
import os
import json
import numpy as np
import matplotlib.pyplot as plt


In [13]:
with open('/home/yonis/histogenomics-msc-2019/yoni-code/MsiPrediction/metadata/tcga/tcga_crc_and_brca_dot_id_to_tcga_id.json') as f:
    dot_id_to_tcga_id = json.load(f)

In [2]:
# TCGA: Logistic: BYOL & SimCLR <-- TP53
# TCGA: SIMCLR DeepMIL & SIMCLR Logistic <-- tertile HRD

In [3]:
# TCGA: BYOL: LOGISTIC: TP53: 69.34%	3.00%
# slurm-6635914_1
byol=["logs/pretrain/1145",
"logs/pretrain/1144",
"logs/pretrain/1143",
"logs/pretrain/1142",
"logs/pretrain/1146"]

# TCGA: SIMCLR: LOGISTIC: TP53: 67.51%	3.58%
# "6775548_1
# 6775615_2-5"
simclr=["logs/pretrain/1395",
"logs/pretrain/1400",
"logs/pretrain/1401",
"logs/pretrain/1399",
"logs/pretrain/1397"]

In [4]:
# TCGA: SIMCLRDEEPMIL: MEDIAN HRD
# "6523037_1 (success): 80.27%	1.73%
# 6523128_2-5 (success)"
deepmil_hrd=["logs/pretrain/771",
"logs/pretrain/773",
"logs/pretrain/772",
"logs/pretrain/774",
"logs/pretrain/775"]

# TCGA: LOGISTIC: MEDIAN HRD
# slurm-6589998_1-5.out: 74.70%	2.45%
log_hrd=["logs/pretrain/952",
'logs/pretrain/953',
"logs/pretrain/955",
"logs/pretrain/956",
"logs/pretrain/954"]


#TCGA : LOGISTIC : TP53
# 69.87%	2.90% slurm-6525915_1-5.out    
log_p53 = [
    "logs/pretrain/819",
    "logs/pretrain/818",
    "logs/pretrain/816",
    "logs/pretrain/817",
    "logs/pretrain/815"
]
#TCGA : DEEPMIL : TP53
# 75.62%	0.95% 6795430_1-5
deepmil_p53 = [
    "logs/eval/1451",
    "logs/eval/1454",
    "logs/eval/1455",
    "logs/eval/1452",
    "logs/eval/1453"
]




In [40]:
root='/home/yonis/SimCLR-1'
ensemble_roc = []
ensemble_predss = [] 
for i, (dirs1, dirs2) in enumerate([[byol, simclr]]):
    print(f" === Combination {i+1} ===")
    for j, (dir1, dir2) in enumerate(zip(dirs1, dirs2)):
        print(f" === Fold {j+1} ===")
        fn1 = [file for file in os.listdir(os.path.join(root, dir1)) if 'regression' in file][0]
        fn2 = [file for file in os.listdir(os.path.join(root, dir2)) if 'regression' in file][0]
        
        df1 = pd.read_csv(os.path.join(root, dir1, fn1))
        df2 = pd.read_csv(os.path.join(root, dir2, fn2))
        
#         for df in [df1, df2]:
#             if df['preds'].dtype == 'float64' # it's a DeepMIL prediction
#                 df['patient2'] = df.apply(lambda x: x, axis=1)
#                 patient_id = dot_id_to_tcga_id[x['patient']].split('-')[2]

        dfg1 = df1.groupby(['patient']).mean()
        patients1 = dfg1.index
#         print(patients1)
        l1 = dfg1['labels'].values
        p1 = np.array(dfg1['preds'].values)
        rocauc1 = metrics.roc_auc_score(y_true=l1, y_score=p1)
        print(rocauc1)
        
        dfg2 = df2.groupby(['patient']).mean()
        patients2 = dfg2.index
#         print(patients2)
        l2 = dfg2['labels'].values
        p2 = np.array(dfg2['preds'].values)
        rocauc2 = metrics.roc_auc_score(y_true=l2, y_score=p2)
        print(rocauc2)
        
        l1 = np.array(l1).astype(np.int16)
        l2 = np.array(l2).astype(np.int16)
        labels=l1
        ensemble_preds = (p1 + p2) / 2
        
        ensemble_predss.append(ensemble_preds)
        
        
        
        ensemble_roc.append(metrics.roc_auc_score(y_true=l1, y_score=ensemble_preds))
    
        
print(np.mean(ensemble_roc), np.std(ensemble_roc))

plt.title('Receiver Operating Characteristic')
for ens_prd in ensemble_preds:
    fpr, tpr, threshold = metrics.roc_curve(labels, ens_prd)
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % rocauc)

    



plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])

plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

plt.legend(loc = 'lower right')
plt.show()
        

 === Combination 1 ===
 === Fold 1 ===
0.6839440321583179
0.6728896103896104
 === Fold 2 ===
0.6705705009276438
0.6783395176252319
 === Fold 3 ===
0.7309059987631417
0.7265383426097712
 === Fold 4 ===
0.7188852813852813
0.672039270253556
 === Fold 5 ===
0.6628014842300557
0.6256184291898579
0.6976035868893012 0.028661119819198114


TypeError: Singleton array 0.01 cannot be considered a valid collection.